In [1]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))

library(tidyverse)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──


In [2]:
CLIN_DIR <- paste0(E_DIR, "/val_hebron/clinical/clean/")
RNA_DIR <- paste0(E_DIR, "/val_hebron/rna/clean/")
TMB_DIR <- paste0(E_DIR, "/val_hebron/somatic/clean/")
ANNOT_DIR <- paste0(E_DIR, "/val_hebron/rna/raw/")
O_DIR <- paste0(E_DIR, "/val_hebron/all_ready/")

### 0 - Read Data Files

In [3]:
clinical <- readRDS( paste0(CLIN_DIR, "clinical_go.Rds"))
somatic <-  readRDS( paste0(TMB_DIR,  "tmb_go.Rds"))
rna <-      readRDS( paste0(RNA_DIR,  "rna_go.Rds"))
annotations <- read.csv( paste0(ANNOT_DIR, "annotations_share.csv"), sep = ";", stringsAsFactors = FALSE)

### 1 - Combine everything

In [4]:
vhio_data <- 
(
clinical 
    %>% inner_join( rna, by = "patient_id")
    %>% left_join( annotations, by = "LAB.ID")
    %>% left_join( somatic, by = "LAB.ID")
)

In [5]:
vhio_select <- (
    vhio_data 
        %>% transmute( 
            patient_id, 
            bor,
            os = ifelse( os_event == 0, -os_days, os_days),
            os_event,
            os_days,
            age = clinical_age, 
            gender = clinical_gender, 
            tissue = Cohort.x,
            tissue_full = Cohort.x, 
            tmb, 
            tcell,
            prolif, 
            tgfb,
            pdl1, 
            pretreat = clinical_pretreat,
            pretreat_comp = NA,
            purity = NA,
            Study = "VHIO"
        ) %>% drop_na(os_days)
          %>% group_by(tissue)
          %>% mutate(tmb=ifelse(is.na(tmb),median(tmb,na.rm=TRUE),tmb))  ### add median imputation
          %>% ungroup()
)

### 2 - Send it

In [6]:
saveRDS( vhio_select, paste0(TMP_DIR, "validation-vhio-go.Rds"))